# EDA

In [81]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/날씨 콘테스트/2012_2016_.csv')
df_hospital = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/날씨 콘테스트/back_hospital.csv')

# dff = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/날씨 콘테스트/2012_2016_.csv')
df_hospital_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/날씨 콘테스트/2-2_검증데이터셋.csv', encoding='CP949')

In [82]:
# 쓸모 없는 column 삭제
# df.drop(columns='Unnamed: 0', inplace=True) 


In [83]:
# 2015, 2016 분리
index_2015 = df[df['YMD'] == '2015-01-01'].index[0] 
index_2016 = df[df['YMD'] == '2016-01-01'].index[0]

df_train = df.iloc[:index_2015, : ] # 2012-2014
df_val = df.iloc[index_2015:index_2016, : ] # 2015
df_test = df.iloc[index_2016:, : ] # 2016

In [84]:
# YMD 형식으로 변경 - 한 번만 돌릴 것!! - hospital 용
def change_str(col):
  col = str(col)
  return col[:4] + '-' + col[4:6] + '-' + col[6:]

# column name 변경
def change_col(df):
  df.rename(columns={'back_hospital.yyyymmdd':'YMD', 'back_hospital.area':'area'}, inplace=True)
  df['YMD'] = df['YMD'].apply(change_str)
  df['YMD'].astype(str) # df도 해주기
  return df
  
def change_col_test(df):
  df.rename(columns={'yyyymmdd':'YMD', 'back_hospital.area':'area'}, inplace=True)
  df['YMD'] = df['YMD'].apply(change_str)
  df['YMD'].astype(str) # df도 해주기
  return df

In [85]:
df_hospital = change_col(df_hospital)
df_hospital_test = change_col_test(df_hospital_test)

In [86]:
df_hospital

,Unnamed: 0,YMD,area,back_hospital.sex,back_hospital.frequency
0,1,2012-01-01,강원,1,3
1,2,2012-01-01,강원,2,3
2,3,2012-01-01,경기,1,4
3,4,2012-01-01,경기,2,5
4,5,2012-01-01,경남,1,2
...,...,...,...,...,...
49669,49670,2015-12-31,제주,2,0
49670,49671,2015-12-31,충남,1,0
49671,49672,2015-12-31,충남,2,0
49672,49673,2015-12-31,충북,1,1


In [87]:
# hospital set 2012-2014, 2015 분리
index_2015 = df_hospital[df_hospital['YMD'] == '2015-01-01'].index[0] 

df_hospital_train = df_hospital.iloc[:index_2015, : ] # 2012-2014
df_hospital_val = df_hospital.iloc[index_2015:, : ] # 2015


In [88]:
print(df_train.shape, df_val.shape, df_test.shape) # 예보 데이터
print(df_hospital_train.shape, df_hospital_val.shape, df_hospital_test.shape) # frequency


(18632, 14) (6205, 14) (6222, 14)
(37264, 5) (12410, 5) (12444, 4)


In [89]:
df_hospital_val.head()
# print(len(df_hospital_val))

,Unnamed: 0,YMD,area,back_hospital.sex,back_hospital.frequency
37264,37265,2015-01-01,강원,1,4
37265,37266,2015-01-01,강원,2,2
37266,37267,2015-01-01,경기,1,4
37267,37268,2015-01-01,경기,2,6
37268,37269,2015-01-01,경남,1,6


In [90]:
# train, val 데이터셋 맞춰주기
def copy_row(mydf):

  mydf = mydf.append(mydf, ignore_index=True)
  mydf.sort_values(by=['YMD', 'area'], inplace=True)
  mydf.reset_index(inplace=True)
  
  return mydf


In [91]:
df_train = copy_row(df_train)
df_val = copy_row(df_val)

In [92]:
print(df_train.shape, df_val.shape, df_test.shape) # 예보 데이터
print(df_hospital_train.shape, df_hospital_val.shape, df_hospital_test.shape) # frequency


(37264, 15) (12410, 15) (6222, 14)
(37264, 5) (12410, 5) (12444, 4)


In [93]:
# test 데이터 맞춰주기
df_test_real = pd.DataFrame()
for i in range(int(len(df_test) / 17)):
  i *= 17
  df_test_real = df_test_real.append(df_test.iloc[i:i+17, :], ignore_index=True) # inplace 없으니 따로 지정해줄 것
  df_test_real = df_test_real.append(df_test.iloc[i:i+17, :], ignore_index=True)

df_test = df_test_real # 원상 복귀

In [94]:
print(df_train.shape, df_val.shape, df_test.shape) # 예보 데이터
print(df_hospital_train.shape, df_hospital_val.shape, df_hospital_test.shape) # frequency


(37264, 15) (12410, 15) (12444, 14)
(37264, 5) (12410, 5) (12444, 4)


In [95]:
# concat 위한 index 초기화
df_hospital_val.reset_index(drop=True, inplace=True)


In [96]:
# df 합치기 
def combine(df_hospital, df):

  df = pd.concat([df_hospital, df], axis=1, join='outer', ) 
  return df

In [97]:
df_train = combine(df_hospital_train, df_train)
df_val = combine(df_hospital_val, df_val) # 이것만 이상하게 나온 이유가 index 0부터 시작 아니네..
df_test = combine(df_hospital_test, df_test)

In [98]:
# 쓸모 없는 column 제거
df_train = pd.concat([df_train.iloc[:, 3:5], df_train.iloc[:, 7:]], axis=1, join='outer') # 새로운 방법 찾아보는 건 좋은데, 시간 오래 걸려도 비효율이다. 선택 잘하자ㄹ
df_val = pd.concat([df_val.iloc[:, 3:5], df_val.iloc[:, 7:]], axis=1, join='outer')
df_test = pd.concat([df_test.iloc[:, 2:4], df_test.iloc[:, 5:]], axis=1, join='outer')



In [99]:
# 결측치 비율 확인
def null_per(df):
  
  return df.isnull().sum() / len(df) * 100 

In [100]:
print(null_per(df_train), '\n') # 6시간 강수량, 적설량, 풍속 결측치 많은데 imputer 쓸 예정
print(null_per(df_val), '\n')
print(null_per(df_test))

back_hospital.sex           0.000000
back_hospital.frequency     0.000000
YMD                         0.000000
area                        0.000000
3시간기온                       5.565693
6시간강수량                     26.835552
6시간신적설                     26.835552
강수형태                        5.565693
강수확률                        5.565693
습도                          5.565693
일최고기온                       5.565693
일최저기온                       5.565693
풍속                         53.515457
풍향                          8.989910
하늘상태                       19.525547
dtype: float64 

back_hospital.sex          0.000000
back_hospital.frequency    0.000000
YMD                        0.000000
area                       0.000000
3시간기온                      2.739726
6시간강수량                     2.739726
6시간신적설                     2.739726
강수형태                       2.739726
강수확률                       2.739726
습도                         2.739726
일최고기온                      2.739726
일최저기온                      2.739

In [101]:
# 결측 비율 공통된거 삭제 - only train, val 
def clear_null(df):
  filter= df['3시간기온'].dropna(axis=0).index
  df = df.iloc[filter, :]
  return df

In [102]:
df_train = clear_null(df_train)
df_val = clear_null(df_val)

In [103]:
# df['YMD'] = pd.to_datetime(df['YMD'], format='%Y-%m-%d') # 오류남 - dtype('<M8[ns]') 랑 머신러닝이랑 안 맞나,,
# df['YMD'] = df['YMD'].values.astype('datetime64[D]') # 이것도 안됨

from datetime import datetime, date, time
## 일단 YMD로 feature 진행

df_train['YMD'] = pd.to_datetime(df_train['YMD'], format='%Y-%m-%d')
df_val['YMD'] = pd.to_datetime(df_val['YMD'], format='%Y-%m-%d')
df_test['YMD'] = pd.to_datetime(df_test['YMD'], format='%Y-%m-%d')

df_train['month'] = df_train['YMD'].dt.month 
df_val['month'] = df_val['YMD'].dt.month 
df_test['month'] = df_test['YMD'].dt.month 

# df_train['year'] = df_train['YMD'].dt.year # year은 중요한 피쳐가 아닌듯..


In [104]:
def set_season(month):
  if 3 <= month <= 5 :
    return 'spring'
  elif 6 <= month <= 8 :
    return 'summer'  
  elif 9 <= month <= 11 :
    return 'fall'  
  elif  (month <= 2) or (month == 12) :
    return 'winter'   

df_train['season'] = df_train['month'].apply(set_season)
df_val['season'] = df_val['month'].apply(set_season)
df_test['season'] = df_test['month'].apply(set_season)


In [105]:
def what_day(date):
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'] 
    day = date.weekday() # 들어온 date(YYYY_MM_DD).weekday() -> 0-6까지 매핑된 결과로 출력 
    return days[day] # days : 월-일 있는 리스트

df_train['day'] = df_train['YMD'].dt.date.apply(what_day) 
df_val['day'] = df_val['YMD'].dt.date.apply(what_day) 
df_test['day'] = df_test['YMD'].dt.date.apply(what_day) 


In [106]:
def is_it_weekend(day):
  if day in ['Sat', 'Sun']:
    return 'Weekend'
  else : 
    return 'Weekdays'

df_train['is_it_weekend'] = df_train['day'].apply(is_it_weekend)
df_val['is_it_weekend'] = df_val['day'].apply(is_it_weekend)
df_test['is_it_weekend'] = df_test['day'].apply(is_it_weekend) # 이것조차도 반복문 쓸까

In [107]:
df_train['YMD'].describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                   35190
unique                   1035
top       2012-01-01 00:00:00
freq                       34
first     2012-01-01 00:00:00
last      2014-12-31 00:00:00
Name: YMD, dtype: object

In [108]:
df_train.rename(columns={'back_hospital.sex':'sex', 'back_hospital.frequency':'frequency'}, inplace=True)
df_val.rename(columns={'back_hospital.sex':'sex', 'back_hospital.frequency':'frequency'}, inplace=True)


In [109]:
# split 전 전체 데이터
# my_df = pd.concat([df_train, df_val, df_test])
# my_df.to_csv('/content/drive/MyDrive/Colab Notebooks/날씨 콘테스트/all_df.csv' )

In [110]:
"""
## YMD 시계열 column drop
df_list = [df_train, df_val, df_test]

for i in df_list:
  i.drop(columns='YMD', inplace=True)
  """

"\n## YMD 시계열 column drop\ndf_list = [df_train, df_val, df_test]\n\nfor i in df_list:\n  i.drop(columns='YMD', inplace=True)\n  "

In [111]:
# val set split
from sklearn.model_selection import train_test_split

target = 'frequency'
features = df_train.columns.drop(target) # 이미 array type

X_train, y_train = df_train.loc[:, features], df_train.loc[:, target]
X_val, y_val = df_val.loc[:, features], df_val.loc[:, target]
X_test, y_test = df_test.loc[:, features], df_test.loc[:, target]

 


In [112]:
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(35190, 18) (12070, 18) (12444, 18) (35190,) (12070,) (12444,)


In [113]:
# csv 출력
# df.to_csv('/content/drive/MyDrive/Colab Notebooks/날씨 콘테스트/total.csv' )

In [114]:
import datetime

now = 1442154124

def timestamp_to_datetime(col):
  return datetime.datetime.fromtimestamp(now).strftime(col)

# sex, month -> float 으로
X_list = [X_train, X_val, X_test]

for i in X_list:
  i[['sex', 'month', 'YMD']] = i[['sex', 'month', 'YMD']].astype(str) # 복사본 반환이라서 새로 지정해줘야 함

  i['YMD'] = i['YMD'].apply(timestamp_to_datetime)


In [137]:
y_val.iloc[:, 0]

IndexingError: ignored

In [129]:
y_list = [y_train, y_val]

for i in y_list:
  i = i.astype(float) # 복사본 반환이라서 새로 지정해줘야 함



In [73]:
print(X_train.dtypes, '\n\n', y_train.dtypes)

sex              object
YMD              object
area             object
3시간기온            object
6시간강수량           object
6시간신적설           object
강수형태             object
강수확률             object
습도               object
일최고기온            object
일최저기온            object
풍속               object
풍향               object
하늘상태             object
month            object
season           object
day              object
is_it_weekend    object
dtype: object 

 frequency    int64
dtype: object


# Model


In [43]:
import tensorflow as tf

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# from tensorflow.keras.layers import Dense, Embedding

# from keras.optimizers import RMSprop

In [45]:
len(X_train.columns)

18

In [46]:
# LSTM 모델을 제작합니다.
import tensorflow as tf

# model을 정의합니다.
"""
keras의 기본 Embedding 벡터를 사용하였으며
LSTM 층에 dropout/recurrent_dropout을 적용하였습니다.
"""

model = Sequential()
model.add(tf.keras.layers.LSTM(64,return_sequences=True, input_shape=(len(X_train), len(X_train.columns))))
model.add(tf.keras.layers.Dropout((0.5)))
# model.add(tf.keras.layers.LSTM(20,return_sequences=False))
# model.add(tf.keras.layers.Dropout((0.5))
model.add(tf.keras.layers.Dense(1)) # 회귀니까 node 1, 활성화 함수 없음
model.compile(loss='rmse', optimizer='rmsprop')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 35190, 64)         21248     
                                                                 
 dropout (Dropout)           (None, 35190, 64)         0         
                                                                 
 dense (Dense)               (None, 35190, 1)          65        
                                                                 
Total params: 21,313
Trainable params: 21,313
Non-trainable params: 0
_________________________________________________________________


In [66]:
model.fit(X_train,y_train, epochs = 100, batch_size = 150, verbose = 0) 


ValueError: ignored

In [60]:
my_unicorns = model.fit(X_train, y_train,
          batch_size=128,
          epochs=60)

ValueError: ignored

In [58]:
unicorns = model.fit(X_train, y_train,
          batch_size=128, 
          epochs=5, 
          validation_data=(X_val,y_val))

ValueError: ignored

In [44]:
# 파라미터를 설정합니다.
max_features = 20000
maxlen = 80
batch_size = 32

In [ ]:
"""
model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(max_features, 128), # Embedding Layer를 거친 후의 shape : (batch_size, maxlen, embedding_size=128)
  tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2), # LSTM Layer를 거친 후의 shape : (batch_size, 1, hidden_size=128)
  tf.keras.layers.Dense(1, activation='sigmoid')
])


"""